# Using DASK

In [27]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.multiprocessing import get
import multiprocessing


## Process Chicago Divvy data 

In [4]:
import pandas as pd
divvy_aggregate = pd.read_csv('./divvy_aggregate_new.csv')
divvy_aggregate.head()
divvy_aggregate['latitudeStart']= divvy_aggregate['latitudeStart'].astype('float64')

### Function to convert latitude longitude to Community Names

In [11]:
import gps_to_neighborhood as gps
all_neighborhoods = gps.get_all_neighborhoods()
def find_nbr(lon,lat):
    #print(lon_lat)
    return gps.find_neighborhood(lon,lat,all_neighborhoods)
find_nbr(-87.616743,41.880958)

'LOOP'

In [28]:
multiprocessing.cpu_count()

12

### Convert latitude longitdue to Community

In [29]:
%%time
data = divvy_aggregate[['longitudeStart','latitudeStart']]
ddata = dd.from_pandas(data, npartitions=4*multiprocessing.cpu_count())

with ProgressBar():
    res = ddata.map_partitions(lambda df: df.apply((lambda row: find_nbr(*row)), axis=1)).compute(scheduler='processes') 

[########################################] | 100% Completed | 40min  6.2s
Wall time: 41min 4s


In [30]:
res.shape

(554434,)

In [32]:
divvy_aggregate['CommunityName'] = res

In [33]:
divvy_aggregate.head()

,Dt,latitudeStart,longitudeStart,AvgTemp,TotalTrips,CommunityName
0,2014-01-01,41.849527,-87.640591,21.0,1,"ARMOUR SQUARE,CHINATOWN"
1,2014-01-01,41.885833,-87.641382,21.0,1,WEST LOOP
2,2014-01-01,41.889180,-87.627700,21.0,1,RIVER NORTH
3,2014-01-01,41.895769,-87.677220,21.0,2,"WICKER PARK,WEST TOWN"
4,2014-01-01,41.895966,-87.667747,17.1,1,"WICKER PARK,WEST TOWN"


**Save Community names dataframe to disk**

In [48]:
divvy_aggregate.to_csv('Divvy_WithCommunityName_Dask.csv',index=False)

### Group by Date and Community Name

In [39]:
divvy_agg_1 = divvy_aggregate.groupby(['Dt','CommunityName']).agg({'AvgTemp':'mean','TotalTrips':'sum'})

In [45]:
divvy_agg_1 = divvy_agg_1.reset_index()

In [46]:
divvy_agg_1.head()

,Dt,CommunityName,AvgTemp,TotalTrips
0,2014-01-01,"ARMOUR SQUARE,CHINATOWN",21.000000,1
1,2014-01-01,BUCKTOWN,21.900000,3
2,2014-01-01,LAKE VIEW,18.587500,8
3,2014-01-01,LINCOLN PARK,19.448571,11
4,2014-01-01,LOGAN SQUARE,21.066667,4


**Total trips from 01-Jan-2014 to 31-12-2017**

In [53]:
divvy_agg_1.TotalTrips.sum()

9058068

**Save Aggregated Data to Disk**

In [52]:
divvy_agg_1.to_csv('DivvyAggregatedBY_Date_Community.csv',index=False)

**Save distinct locations and their count to disk**

In [51]:
ser = divvy_agg_1['CommunityName'].value_counts()
ser.to_csv('CommunityNamesDivvy.csv',index=True)

C:\ProgramData\Anaconda3\envs\daskenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


## Consolidated Code

In [ ]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.multiprocessing import get
import multiprocessing
import pandas as pd


divvy_aggregate = pd.read_csv('./divvy_aggregate_new.csv')
divvy_aggregate.head()
divvy_aggregate['latitudeStart']= divvy_aggregate['latitudeStart'].astype('float64')


import gps_to_neighborhood as gps
all_neighborhoods = gps.get_all_neighborhoods()
def find_nbr(lon,lat):
    #print(lon_lat)
    return gps.find_neighborhood(lon,lat,all_neighborhoods)
find_nbr(-87.616743,41.880958)

#%%time
data = divvy_aggregate[['longitudeStart','latitudeStart']]
ddata = dd.from_pandas(data, npartitions=4*multiprocessing.cpu_count())

with ProgressBar():
    res = ddata.map_partitions(lambda df: df.apply((lambda row: find_nbr(*row)), axis=1)).compute(scheduler='processes')
    
res.shape
divvy_aggregate['CommunityName'] = res
divvy_aggregate.to_csv('Divvy_WithCommunityName_Dask.csv',index=False)
divvy_agg_1 = divvy_aggregate.groupby(['Dt','CommunityName']).agg({'AvgTemp':'mean','TotalTrips':'sum'})
divvy_agg_1 = divvy_agg_1.reset_index()
print('Total Trips:',divvy_agg_1.TotalTrips.sum())
divvy_agg_1.to_csv('DivvyAggregatedBY_Date_Community.csv',index=False)

ser = divvy_agg_1['CommunityName'].value_counts()
ser.to_csv('CommunityNamesDivvy.csv',index=True)